In [5]:
from dotenv import load_dotenv
load_dotenv()
import os #to access the environment variables
import pinecone
from pinecone import Pinecone, ServerlessSpec
from llama_index.llms.gemini import Gemini
from llama_index.vector_stores.pinecone import PineconeVectorStore
from llama_index.embeddings.gemini import GeminiEmbedding
from llama_index.core import StorageContext, VectorStoreIndex, download_loader
import google.generativeai as genai
from llama_index.core import Settings


In [16]:
#creating db instance
pc = Pinecone(api_key=os.getenv('PINECONE_API_KEY'))

#creating index on pinecone (comment out when using locally)
# pc.create_index(
#     name="rag-rmp",
#     dimension=768,
#     metric="cosine",
#     spec=ServerlessSpec(cloud="aws", region="us-east-1"),
# )

In [7]:
#reading the reviews file
import json
data = json.load(open("reviews.json"))
# data['reviews']

In [12]:
#By default, LlamaIndex assumes you're using OpenAI to generate embeddings. To configure it to use Gemini instead, we need to set up the service context which lets LlamaIndex know which llm and which embedding model to use.
os.environ["GOOGLE_API_KEY"] = os.getenv('GEMINI_API_KEY')
genai.configure(api_key=os.environ["GOOGLE_API_KEY"])
client = Gemini()
# embed_model = GeminiEmbedding(
#     model_name="models/embedding-004"
# )

# Settings.embed_model = embed_model
# Settings.llm = client
# Settings.chunk_size = 512

processed_data = []
for review in data["reviews"]:
    response = genai.embed_content(
        model="models/text-embedding-004",
        content=review["review"],
        task_type="retrieval_document",
        title='review of '+review["professor_name"] 
    )
   
    embedding = response["embedding"]
    processed_data.append({
        "values":embedding,
        "id": review["professor_name"],
        "metadata":{
            "subject": review["subject"],
            "domain": review["domain"],
            "rating": review["rating"],
            "review": review["review"],
        }
    })

In [ ]:
processed_data[0]

In [17]:
#creating index and upsert (insert into db) upsert operation writes vectors into a namespace. If a new value is upserted for an existing vector id, it will overwrite the previous value.
index = pc.Index('rag-rmp') #collection
index.upsert(
    vectors=processed_data,
    namespace="namespace1" #document
)


{'upserted_count': 40}

In [21]:
#index description
index.describe_index_stats()
#{'dimension': 768,
#'index_fullness': 0.0,
# 'namespaces': {'namespace1': {'vector_count': 40}},
# 'total_vector_count': 40}


{'dimension': 768,
 'index_fullness': 0.0,
 'namespaces': {'namespace1': {'vector_count': 40}},
 'total_vector_count': 40}